<a href="https://colab.research.google.com/github/maybachar/getpet-recognition/blob/master/training_model_cat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from glob import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from tensorflow.keras.applications import DenseNet169
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from keras.models import model_from_json

# num of breeds and image size.
num_breeds = 12
size = 224


def build():
    # The input size the model will get. default size for mobile and 3 is for colors.
    inputs_model = Input((size, size, 3))
    # create the base pre-trained model.
    base_model = DenseNet169(input_tensor=inputs_model, include_top=False,weights="imagenet", input_shape=(size, size, 3))
    # set all parameters to be trainable
    base_model.trainable = True
    # get the model so we can add layers to it.
    model_out = base_model.output
    # add a global spatial average pooling layer
    model_out = GlobalAveragePooling2D()(model_out)
    # include a dropout layer to minimize the overfitting.
    model_out = Dropout(0.2)(model_out)
    # using activation function RELU- widely used in CNN.
    model_out = Dense(1024, activation="relu")(model_out)
    # add softmax layer for getting probabilities on the breeds.
    model_out = Dense(num_breeds, activation="softmax")(model_out)
    # create the model with it's layers.
    model = tf.keras.Model(inputs_model, model_out)
    return model


def fix_data_element(path, y):
    # one hot encoding to lable (all zero except for the place y).
    one_hot_y = [0] * num_breeds
    one_hot_y[y] = 1
    one_hot_y = np.array(one_hot_y)
    # Cast to type as defined.
    one_hot_y = one_hot_y.astype(np.int32)
    path = path.decode()
    # loads a color image from the specified file.
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    # change the width, height of an image.
    img = cv2.resize(img, (size, size))
    # normaliztion.
    img = img / 255.0
    img = img.astype(np.float32)
    return img, one_hot_y

def wrap_elements(x, y):
    # wrap numpy function as an operation in TensorFlow function.
    x, y = tf.numpy_function(fix_data_element, [x, y], [tf.float32, tf.int32])
    # resize image to the image size mobilenet expect to get.
    x.set_shape((size, size, 3))
    y.set_shape((num_breeds))
    return x, y

def fix_dataset(x, y, batch):
    # get the slices of x and y into one dataset.
    data_slices = tf.data.Dataset.from_tensor_slices((x, y))
    # execute fix_data function on every element of the Dataset separately.
    data_slices = data_slices.map(wrap_elements)
    # combines consecutive elements of a dataset object into batches.
    data_slices = data_slices.batch(batch)
    return data_slices



if __name__ == "__main__":
    # learning rate 0.0001.
    lr = 0.0001
    batch = 16
    epochs = 3
    # paths of train data folder and csv file with labels of each pic.
    path = "/content/drive/MyDrive/cat-breed-identification/"
    train_path = os.path.join(path, "images/images/*")
    labels_path = os.path.join(path, "cat_labels.csv")
    # import the CSV file.
    labels_file = pd.read_csv(labels_path)
    # get all unique breeds in file.
    breed = labels_file["breed"].unique()
    length_breed=len(breed)
    breed_num_dic = {}
    # create dictionary of breed and number.
    for i in range(length_breed):
      breed_num_dic[breed[i]] = i
    # ids of photos
    all_img_ids = glob(train_path)
    # list of breed index by order of image id
    img_lables = []
    # Append to lables list the suitale lable for each image using info from csv file
    # and add the number of breed instead the name breed(using dictionary).
    for img_id in all_img_ids:
        img_id = img_id.split("/")[-1]
        img_id = img_id.split(".")[0]
        # get breed name by image id from labels.csv
        breed_name = list(labels_file[labels_file.id == img_id]["breed"])[0]
        # get the number of this breed and append it to list.
        breed_idx = breed_num_dic[breed_name]
        img_lables.append(breed_idx)
    # work only with N image id and N lables of the breed id which suitable to the images.
    all_img_ids = all_img_ids[:2350]
    img_lables = img_lables[:2350]
    # spliting data to train and validation.
    train_x, validation_x = train_test_split(all_img_ids, test_size=0.2, random_state=42)
    train_y, validation_y = train_test_split(img_lables, test_size=0.2, random_state=42)
    # dataset
    trainSet = fix_dataset(train_x, train_y, batch=batch)
    validationSet = fix_dataset(validation_x, validation_y, batch=batch)
    #create a CNN model with it's layers.
    model = build()
    #Configures the model for training. optimizer- adam.
    model.compile(loss="categorical_crossentropy", optimizer=Adam(lr), metrics=["acc"])
    # Train
    #set of functions to be applied at training procedure:
    # modelCheckPoint saves the model after every epoch.
    # Reduce learning rate when a metric has stopped improving.
    callbacks = [ModelCheckpoint("model.h5", verbose=1, save_best_only=True),
        ReduceLROnPlateau(factor=0.1, patience=2, min_lr=0.000001)]
    # Trains the model for number of epochs. evaluate the loss at the end of each epoch.
    model.fit(trainSet, validation_data=validationSet, epochs=epochs, callbacks=callbacks)
    # save the model and it's weights in H5 format to jeson file.
    model_json = model.to_json()
    with open("/content/drive/MyDrive/cat-breed-identification/model.json", "w") as json_file:
        json_file.write(model_json)
        model.save_weights("/content/drive/MyDrive/cat-breed-identification/model.h5")
        print("Saved model to disk")

